In [1]:
import os
os.environ["MANTLE"] = "lattice"
import silica as si
import magma as m
from silica import bits, add
from magma.bit_vector import BitVector
from magma.testing.coroutine import check

def PISO(n):
    @si.coroutine(inputs={"PI": si.Bits(n), "SI": si.Bit, "LOAD": si.Bit})
    def PISO():
        values = bits(0, n)
        O = values[-1]
        while True:
            PI, SI, LOAD = yield O
            O = values[-1]
            if LOAD:
                values = PI
            else:
                # values = [SI] + values[:-1]
                for i in range(n - 1, 0, -1):
                    values[i] = values[i - 1]
                values[0] = SI
    return PISO()
        
PISO10 = si.compile(PISO(10), file_name="build/silica_piso.py")
with open("build/silica_piso.py", "r") as magma_file:
    print(magma_file.read())

import mantle lattice ice40
import mantle lattice mantle40
from magma import *
import os
os.environ["MANTLE"] = os.getenv("MANTLE", "coreir")
from mantle import *
import mantle.common.operator

PISO = DefineCircuit("PISO", "O", Out(Bit), "PI", In(Bits(10)), "SI", In(Bit), "LOAD", In(Bit), *ClockInterface(has_ce=False))
CE = VCC
Buffer = DefineCircuit("__silica_BufferPISO", "I", In(Bits(2)), "O", Out(Bits(2)))
wire(Buffer.I, Buffer.O)
EndDefine()
__silica_path_state = Buffer()
PI = PISO.PI
SI = PISO.SI
LOAD = PISO.LOAD
values = Register(10, has_ce=True)
values_CE = [CE]
wireclock(PISO, values)
values_next = Or(2, 10)
wire(values_next.O, values.I)
values_next_0 = And(2, 10)
wire(values_next_0.O, values_next.I0)
wire(values_next_0.I0[0], __silica_path_state.O[0])
wire(values_next_0.I0[1], __silica_path_state.O[0])
wire(values_next_0.I0[2], __silica_path_state.O[0])
wire(values_next_0.I0[3], __silica_path_state.O[0])
wire(values_next_0.I0[4], __silica_path_state.O[0])
wire(values_next_0.I0

In [2]:
!magma -o coreir -m coreir -t PISO build/silica_piso.py
!coreir -i build/silica_piso.json -o build/silica_piso.v
!yosys -p 'synth_ice40 -top PISO -blif build/silica_piso.blif' build/silica_piso.v | grep -A 14 "2.27. Printing statistics."
!arachne-pnr -q -d 1k -o build/silica_piso.txt build/silica_piso.blif
!icetime -tmd hx1k build/silica_piso.txt | grep -B 2 "Total path delay"

In Run Generators

Modified?: Yes
2.27. Printing statistics.

=== PISO ===

   Number of wires:                309
   Number of wire bits:            538
   Number of public wires:         309
   Number of public wire bits:     538
   Number of memories:               0
   Number of memory bits:            0
   Number of processes:              0
   Number of cells:                 20
     SB_DFF                         10
     SB_LUT4                        10


Total number of logic levels: 1
Total path delay: 2.23 ns (448.01 MHz)


In [3]:
import os
os.environ["MANTLE"] = "lattice"
from magma import compile
from mantle import DefinePISO
compile("build/mantle_piso", DefinePISO(10))

compiling Mux2
compiling Register10
compiling PISO10


In [4]:
!yosys -p 'synth_ice40 -top PISO10 -blif build/mantle_piso.blif' build/mantle_piso.v | grep -A 14 "2.27. Printing statistics."
!arachne-pnr -q -d 1k -o build/mantle_piso.txt build/mantle_piso.blif
!icetime -tmd hx1k build/mantle_piso.txt | grep -B 2 "Total path delay"

2.27. Printing statistics.

=== PISO10 ===

   Number of wires:                 69
   Number of wire bits:            115
   Number of public wires:          69
   Number of public wire bits:     115
   Number of memories:               0
   Number of memory bits:            0
   Number of processes:              0
   Number of cells:                 20
     SB_DFF                         10
     SB_LUT4                        10


Total number of logic levels: 1
Total path delay: 2.17 ns (461.05 MHz)
